# 7. Simulations with increasing cell counts

To assess the effect of cell count and the number of bootstrapped samples on the ability to identify significant clusters by scanpro, we increment the `mu` parameter of the function `simulate_cell_counts_2`, which result in increased cell count each time. For each `mu` parameter, we generate 100 datasets randomly and run scanpro_boot on each, and the mean cell count is then calculated.

In [1]:
import pandas as pd
import numpy as np
import time
import multiprocessing as mp

from scanpro import scanpro
from scanpro.utils import estimate_params_from_counts, simulate_cell_counts_2, convert_counts_to_df

In [2]:
OUT_PATH = 'results/benchmark'

In [3]:
def generate_dataset(p, a, b, n_reps, n_conds=2, n=20, mu=5000, seed=1):

    np.random.seed(seed)
    if b is None:
        b = a * (1-p) / p

    counts = simulate_cell_counts_2(p, n_reps, a, b, n_conds, n=n, mu=mu)

    counts_df = convert_counts_to_df(counts, column_name="cluster")

    return counts_df

In [4]:
def simulate_datasets(p, a, b, mu, n_reps=4, n_sims=100):
    np.random.seed(10)
    
    pool = mp.Pool()
    jobs = []
    
    for sim in range(n_sims):

        # Run Scanpro or scanpro bootstrapping on 100 datasets
        job = pool.apply_async(generate_dataset, (p, a, b, n_reps), dict(n_conds=2, n=20,
                                                                         mu=mu, seed=sim))
        jobs.append(job)

    pool.close()
    monitor_jobs(jobs)

    results = [job.get() for job in jobs]
    pool.join()

    return results

In [5]:
def monitor_jobs(jobs):
    """
    Monitor the status of jobs submitted to a pool.

    Parameters
    ----------
    jobs : list of job objects
        List of job objects, e.g. as returned by pool.map_async().
    """

    if isinstance(jobs, dict):
        jobs = list(jobs.values())

    from tqdm import tqdm_notebook as tqdm
    pbar = tqdm(total=len(jobs))
    
    # Wait for all jobs to finish
    n_ready = sum([job.ready() for job in jobs])
    while n_ready != len(jobs):
        if n_ready != pbar.n:
            pbar.n = n_ready
            pbar.refresh()
        time.sleep(1)
        n_ready = sum([job.ready() for job in jobs])

    pbar.n = n_ready  # update progress bar to 100%
    pbar.refresh()
    pbar.close()

In [6]:
def test_performance(datasets,
                     n_reps,  # number of samples per condition
                     transform,
                     ):
    """Test the performance of bootstrap scanpro on simulated data.
    :param list datasets: List of datasets as pandas dataframes to run scanpro on
    :param int n_reps: Number of replicates the bootstrap is going to generate
    :param str transform: method of transformation (logit or arcsin)
    :return pandas.DataFrame all_run_results: A dataframe with results from all runs.
    """
    import warnings
    warnings.filterwarnings('ignore')

    jobs = []

    for mu in datasets.keys():
        pool = mp.Pool()
        for dataset in datasets[mu]:

            # Run Scanpro or scanpro bootstrapping on 100 datasets
            job = pool.apply_async(scanpro, (dataset,), dict(clusters_col="cluster", conds_col="group", 
                                                                     samples_col=None, n_reps=n_reps,
                                                                     transform=transform, verbosity=0))
            jobs.append(job)

        pool.close()
        monitor_jobs(jobs)

        results = [job.get() for job in jobs]
        pool.join()

    # Add ID to each result
    for i, result in enumerate(results):
        result.results["run"] = i + 1

    # Collect result
    all_run_results = [result.results for result in results]

    return all_run_results

In [129]:
#### HELPER STATISTCS FUNCTIONS ####

def compare_p_values(p_values, per_cluster=False):
    """ Compare p-values of simulation with differences with expected output"""
    out = []
    for x in p_values:
        if per_cluster:
            # save hit rate per cluster
            hitrate = [x[0]<.05, x[1]>=.05, x[2]>=.05, x[3]<.05, x[4]>=.05, x[5]>=.05, x[6]<0.05]
        else:
            # save hit rate per run
            hitrate = x[0]<0.05 and x[1]>=0.05 and x[2]>=0.05 and x[3]<0.05 and x[4]>=0.05 and x[5]>=0.05 and x[6]<0.05
        out.append(hitrate)

    return out


def get_means(data):
    """Get mean cell count for each mu"""
    return [np.mean([x.shape[0] for x in data[mu]]) for mu in data.keys()]


def get_sample_means(data):
    """Get samples mean cell count for each mu"""
    return [np.mean([x['group'].value_counts().mean() for x in data[mu]]) for mu in data.keys()]


def calc_hitrates(results, compare=None):
    out = {mu: {method: None for method in list(results[mu].keys())} for mu in results.keys()}
    for mu in results.keys():
        for method in results[mu]:
            p_values = [results[mu][method].iloc[i:i+7, 3] for i in np.arange(0, len(results[mu][method]), 7)]

            if compare is None:
                raise ValueError("Please provide a compare function!")
                
            out[mu][method] = np.mean(compare(p_values)) * 100
    
    out = pd.DataFrame(out)
    out = out.reset_index().melt(id_vars='index')
    out.rename(columns={'index': 'method', 'variable': 'mu', 'value': 'hit_rate'}, inplace=True)

    return out


def get_stats(p_values, counts, n_sims):
    """ Calculate sensitivty, specificty and false positive rate """
    from sklearn.metrics import roc_auc_score
    
    methods = list(p_values[counts[0]].keys())
    results = {count: {method: [] for method in methods} for count in counts}
    df_list = []
    # True is significant cell type in original data
    y_test = np.tile([True, False, False, True, False, False, True], n_sims)  
    for i, count in enumerate(counts):
        for method in methods:
            test_df = pd.DataFrame(p_values[count][method])
            test_stat = np.zeros((2,2))
            test_stat[0][0] = sum([sum(test_df.iloc[:,i] < 0.05) for i in [0,3,6]])  # tp
            test_stat[0][1] = sum([sum(test_df.iloc[:,i] < 0.05) for i in [1,2,4,5]])  # fp
            test_stat[1][0] = sum([sum(test_df.iloc[:,i] > 0.05) for i in [0,3,6]])  # fn
            test_stat[1][1] = sum([sum(test_df.iloc[:,i] > 0.05) for i in [1,2,4,5]])  # tn
            
            sens = test_stat[0][0] / test_stat.sum(axis=0)[0]  # true positive rate
            specif = test_stat[1][1] / test_stat.sum(axis=0)[1]  # specificity
            fpr = test_stat[0][1] / test_stat.sum(axis=0)[1]  # false positive rate
            acc = (test_stat[0][0] + test_stat[1][1]) / np.sum(test_stat)  # accuracy (TP + TN) / (TP + TN + FP + FN)
            
            results[count][method].append(sens)
            results[count][method].append(specif)
            results[count][method].append(fpr)
            results[count][method].append(acc)
            
            # calculate auroc
            y_predicted = np.array(p_values[count][method]).flatten()
            y_predicted = y_predicted < 0.05
        
            auroc = roc_auc_score(y_test, y_predicted)
            results[count][method].append(auroc)
        
        tmp = pd.DataFrame(results[count], index=['sensitivity', 'specificity', 'fpr', 'accuracy', 'auroc'])
        tmp = tmp.T
        tmp['mean_replicate_count'] = count
        
        df_list.append(tmp)
    
    df_stat = pd.concat(df_list, axis=0, join='outer')
    df_stat.reset_index(inplace=True)
    df_stat.rename({'index': 'method'}, inplace=True, axis=1)
    df_stat['reps'] = df_stat['method'].str.split('_').str[1]
    df_stat['Transform'] = df_stat['method'].str.split('_').str[0]
    
    return df_stat

## Data preperation
see https://phipsonlab.github.io/propeller-paper-analysis/SimTrueDiff.html

In [8]:
heart_counts = pd.read_csv('data/heart_counts.tsv', sep='\t')
heart_counts.drop(['Condition', 'Sex'], inplace=True, axis=1)
heart_counts = heart_counts.set_index('Sample').T
heart_counts.drop('Erythroid', inplace=True)  # remove erythroids

# proportions of each cluster in all samples
true_props = heart_counts.sum(axis=1) / heart_counts.sum(axis=1).sum()  # sum of cells in cluster / sum of all cells
true_props = true_props.to_frame(name="props")

# estimate beta paramters from counts
params = estimate_params_from_counts(heart_counts)  # rows are clusters
a = params[1]
b = params[2]

# Set up true proportions for the two groups
grp1_trueprops = true_props.values.flatten()
grp2_trueprops = true_props.values.flatten()

grp2_trueprops[0] = grp1_trueprops[0]/2
grp2_trueprops[3] = grp2_trueprops[3]*2
grp2_trueprops[6] = grp1_trueprops[6]*3

grp2_trueprops[0] = grp2_trueprops[0] + (1-grp2_trueprops.sum())/2
grp2_trueprops[3] = grp2_trueprops[3] + (1-grp2_trueprops.sum())
 
# calculate beta for both groups
b1 = a*(1-grp1_trueprops)/grp1_trueprops
b2 = a*(1-grp2_trueprops)/grp2_trueprops
b_grps = [b1, b2]

b_grps

[Cardiomyocytes           1.769390
 Endothelial cells      107.760684
 Epicardial cells        47.297371
 Fibroblast              14.138098
 Immune cells            16.442437
 Neurons                143.134155
 Smooth muscle cells    458.574736
 dtype: float64,
 Cardiomyocytes           4.742325
 Endothelial cells      107.760684
 Epicardial cells        47.297371
 Fibroblast               4.666523
 Immune cells            16.442437
 Neurons                143.134155
 Smooth muscle cells    150.405074
 dtype: float64]

## simulate datasets

In [9]:
n_reps = 4  # number of replicates before merge
mu_list = [250, 500, 750, 1000, 1250, 1500, 2000, 2500, 3500, 5000, 6000, 7000, 9000, 10000, 11000, 12500]

datasets = {f"mu_{mu}": simulate_datasets(true_props, a, b_grps, n_reps=n_reps, mu=mu, n_sims=100) for mu in mu_list}

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [113]:
# get mean cell counts for each mu
mean_counts = get_means(datasets)
# get mean cell counts for samples in each mu
mean_replicate_count = get_sample_means(datasets)

mean_replicate_count

[978.62,
 1972.685,
 2956.855,
 3938.335,
 4917.685,
 5904.13,
 7867.8,
 9837.58,
 13766.055,
 19671.785,
 23601.63,
 27537.03,
 35424.58,
 39346.91,
 43280.78,
 49193.815]

## Test bootstrapping with different cell counts

### 2 replicates

In [11]:
n_reps = 2

#### logit

In [12]:
scanpro_2_reps_logit = test_performance(datasets, transform="logit",
                                        n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

#### arcsine

In [13]:
scanpro_2_reps_arcsin = test_performance(datasets, transform="arcsin",
                                         n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

### 3 replicates

In [14]:
n_reps = 3

#### logit

In [15]:
scanpro_3_reps_logit = test_performance(datasets, transform="logit",
                                        n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

#### arcsine

In [16]:
scanpro_3_reps_arcsin = test_performance(datasets, transform="arcsin",
                                         n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

### 4 replicates

In [17]:
n_reps = 4

#### logit

In [18]:
scanpro_4_reps_logit = test_performance(datasets, transform="logit",
                                        n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

#### arcsine

In [19]:
scanpro_4_reps_arcsin = test_performance(datasets, transform="arcsin",
                                         n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

### 5 replicates

In [20]:
n_reps = 5

#### logit

In [21]:
scanpro_5_reps_logit = test_performance(datasets, transform="logit",
                                        n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

#### arcsine

In [22]:
scanpro_5_reps_arcsin = test_performance(datasets, transform="arcsin",
                                         n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

### 8 replicates

In [23]:
n_reps = 8

#### logit

In [24]:
scanpro_8_reps_logit = test_performance(datasets, transform="logit",
                                        n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

#### arcsine

In [25]:
scanpro_8_reps_arcsin = test_performance(datasets, transform="arcsin",
                                        n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

### 10 reps

In [26]:
n_reps = 10

#### logit

In [27]:
scanpro_10_reps_logit = test_performance(datasets, transform="logit",
                                        n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

#### arcsine

In [28]:
scanpro_10_reps_arcsin = test_performance(datasets, transform="arcsin",
                                        n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

### 14 reps

In [29]:
n_reps = 14

#### logit

In [30]:
scanpro_14_reps_logit = test_performance(datasets, transform="logit",
                                         n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

#### arcsine

In [31]:
scanpro_14_reps_arcsin = test_performance(datasets, transform="arcsin",
                                         n_reps=n_reps)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

  0%|          | 0/1300 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

### Results

In [116]:
methods = [f'{trans}_{rep}_reps' for rep in [2,3,4,5,8,10,14] for trans in ['logit', 'arcsin']]

results = [scanpro_2_reps_logit, scanpro_2_reps_arcsin,
           scanpro_3_reps_logit, scanpro_3_reps_arcsin,
           scanpro_4_reps_logit, scanpro_4_reps_arcsin,
           scanpro_5_reps_logit, scanpro_5_reps_arcsin,
           scanpro_8_reps_logit, scanpro_8_reps_arcsin,
           scanpro_10_reps_logit, scanpro_10_reps_arcsin,
           scanpro_14_reps_logit, scanpro_14_reps_arcsin]

all_results = {mu: {method: None for method in methods} for mu in mu_list}

differential_groups = ['Cardiomyocytes', 'Fibroblast', 'Smooth muscle cells']

for i, method in enumerate(methods):
    for j, mu in enumerate(mu_list):
    
        all_results[mu][method] = pd.concat(results[i][j*100:j*100+100])
        all_results[mu][method]['mean_replicate_count'] = mean_replicate_count[j]
        all_results[mu][method]['mu'] = mu
        all_results[mu][method]['n_reps'] = method.split('_')[1]
        all_results[mu][method]['transform'] = method.split('_')[0]
        all_results[mu][method]['correct'] = [row["p_values"] < 0.05 if row["clusters"] in differential_groups else row["p_values"] >= 0.05 for i, row in all_results[mu][method].reset_index().iterrows()]


In [117]:
all_results_df = pd.concat([all_results[mu][method] for mu in mu_list for method in methods])
all_results_df.head()

,baseline_props,mean_props_cond_1,mean_props_cond_2,p_values,run,mean_replicate_count,mu,n_reps,transform,correct
clusters,,,,,,,,,,
Cardiomyocytes,0.426893,0.531897,0.316813,0.062523,1,978.62,250,2,logit,False
Endothelial cells,0.071689,0.071863,0.069967,0.608524,1,978.62,250,2,logit,True
Epicardial cells,0.062778,0.066618,0.058406,0.525534,1,978.62,250,2,logit,True
Fibroblast,0.298906,0.223336,0.379895,0.080629,1,978.62,250,2,logit,False
Immune cells,0.109761,0.089133,0.129736,0.223243,1,978.62,250,2,logit,True


In [118]:
all_results_df.to_csv(f"{OUT_PATH}/simulation_results_diff_2.tsv", sep="\t")

In [119]:
# get p_values for each method
p_values = {count: {method: None for method in methods} for count in mean_replicate_count}

for method in methods:
    for i, count in enumerate(sample_mean_counts):

        p_values[count][method] = [all_results[mu_list[i]][method].iloc[j:j+7, 3].to_list() for j in np.arange(0, 700, 7)]

#### Hit rate

In [126]:
hit_rates = calc_hitrates(all_results, compare_p_values)
# add mean cell counts
hit_rates['mean_count'] = np.repeat(mean_counts, len(methods))
# add sample mean cell counts
hit_rates['mean_replicate_count'] = np.repeat(mean_replicate_count, len(methods))
# add transofrmation as seperate column
hit_rates['Transform'] = hit_rates['method'].str.split('_').str[0]

hit_rates

,method,mu,hit_rate,mean_count,mean_replicate_count,Transform
0,logit_2_reps,250,0.0,1957.24,978.620,logit
1,arcsin_2_reps,250,5.0,1957.24,978.620,arcsin
2,logit_3_reps,250,0.0,1957.24,978.620,logit
3,arcsin_3_reps,250,4.0,1957.24,978.620,arcsin
4,logit_4_reps,250,0.0,1957.24,978.620,logit
...,...,...,...,...,...,...
219,arcsin_8_reps,12500,34.0,98387.63,49193.815,arcsin
220,logit_10_reps,12500,10.0,98387.63,49193.815,logit
221,arcsin_10_reps,12500,24.0,98387.63,49193.815,arcsin
222,logit_14_reps,12500,3.0,98387.63,49193.815,logit


In [127]:
# save
hit_rates.to_csv(f'{OUT_PATH}/benchmark_hitrates.tsv', sep='\t')

#### specificity, sensitivity and fpr

In [130]:
stats = get_stats(p_values, mean_replicate_count, n_sims=100)
stats

,method,sensitivity,specificity,fpr,accuracy,auroc,mean_replicate_count,reps,Transform
0,logit_2_reps,0.346667,0.9900,0.0100,0.714286,0.668333,978.620,2,logit
1,arcsin_2_reps,0.663333,0.9125,0.0875,0.805714,0.787917,978.620,2,arcsin
2,logit_3_reps,0.396667,1.0000,0.0000,0.741429,0.698333,978.620,3,logit
3,arcsin_3_reps,0.623333,0.9425,0.0575,0.805714,0.782917,978.620,3,arcsin
4,logit_4_reps,0.266667,1.0000,0.0000,0.685714,0.633333,978.620,4,logit
...,...,...,...,...,...,...,...,...,...
219,arcsin_8_reps,0.860000,0.8300,0.1700,0.842857,0.845000,49193.815,8,arcsin
220,logit_10_reps,0.530000,0.9825,0.0175,0.788571,0.756250,49193.815,10,logit
221,arcsin_10_reps,0.723333,0.9250,0.0750,0.838571,0.824167,49193.815,10,arcsin
222,logit_14_reps,0.306667,0.9950,0.0050,0.700000,0.650833,49193.815,14,logit


In [131]:
# save
stats.to_csv(f'{OUT_PATH}/benchmark_stats.tsv', sep='\t')

In [132]:
# get best auroc for each mean count
best_auroc = stats.sort_values('auroc', ascending=False).drop_duplicates('mean_replicate_count').sort_values('mean_replicate_count')
best_auroc

,method,sensitivity,specificity,fpr,accuracy,auroc,mean_replicate_count,reps,Transform
1,arcsin_2_reps,0.663333,0.9125,0.0875,0.805714,0.787917,978.620,2,arcsin
15,arcsin_2_reps,0.796667,0.8100,0.1900,0.804286,0.803333,1972.685,2,arcsin
31,arcsin_3_reps,0.813333,0.7950,0.2050,0.802857,0.804167,2956.855,3,arcsin
42,logit_2_reps,0.863333,0.7575,0.2425,0.802857,0.810417,3938.335,2,logit
61,arcsin_4_reps,0.810000,0.7900,0.2100,0.798571,0.800000,4917.685,4,arcsin
76,logit_5_reps,0.650000,0.9600,0.0400,0.827143,0.805000,5904.130,5,logit
90,logit_5_reps,0.700000,0.9050,0.0950,0.817143,0.802500,7867.800,5,logit
105,arcsin_5_reps,0.846667,0.7700,0.2300,0.802857,0.808333,9837.580,5,arcsin
118,logit_5_reps,0.806667,0.8125,0.1875,0.810000,0.809583,13766.055,5,logit
135,arcsin_8_reps,0.716667,0.9250,0.0750,0.835714,0.820833,19671.785,8,arcsin


In [133]:
# save
best_auroc.to_csv(f'{OUT_PATH}/benchmark_best_auroc.tsv', sep='\t')